In [ ]:
output_chm_path = 'output_chm.tif'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install rasterio numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 56.1 MB/s eta 0:00:00


In [ ]:
import rasterio
from rasterio.enums import Resampling

# Open the DSM to get its spatial characteristics
with rasterio.open('/content/drive/MyDrive/IV Year Major Project/Tree Height/DSM and DTM/Single DSM DTM/210930_Major Project_dsm (1).tif') as dsm_src:
    dsm = dsm_src.read(1)  # Read the first band
    dsm_transform = dsm_src.transform
    dsm_crs = dsm_src.crs

# Open the DTM, but this time we'll resample it
with rasterio.open('/content/drive/MyDrive/IV Year Major Project/Tree Height/DSM and DTM/Single DSM DTM/210930_Major Project_dtm.tif') as dtm_src:
    # Calculate the scaling factors for x and y directions
    scale_x = dsm_src.width / dtm_src.width
    scale_y = dsm_src.height / dtm_src.height

    # Perform resampling
    dtm_resampled = dtm_src.read(
        1,
        out_shape=(
            dtm_src.count,
            int(dtm_src.height * scale_y),
            int(dtm_src.width * scale_x)
        ),
        resampling=Resampling.bilinear  # or choose another resampling method
    )

    # Adjust the transform of the resampled DTM
    new_transform = dtm_src.transform * dtm_src.transform.scale(
        (dtm_src.width / dtm_resampled.shape[-1]),
        (dtm_src.height / dtm_resampled.shape[-2])
    )

# Now dtm_resampled should have the same shape as dsm
chm = dsm - dtm_resampled

# Save the CHM to a new TIFF file, making sure to use the DSM's transform and CRS
with rasterio.open(
    '/content/drive/MyDrive/IV Year Major Project/Tree Height/chm_empty.tif', 'w',
    driver='GTiff',
    height=chm.shape[0],
    width=chm.shape[1],
    count=1,
    dtype=chm.dtype,
    crs=dsm_crs,
    transform=dsm_transform,
) as dst:
    dst.write(chm, 1)

# **QGIS CHM**

In [ ]:
import rasterio
import numpy as np

# Path to your CHM file
chm_path = '/content/drive/MyDrive/IV Year Major Project/Tree Height/chm_dtm_dsm_1.tif'

# Open the CHM file
with rasterio.open(chm_path) as chm_src:
    chm_array = chm_src.read(1)  # Read the first band
    chm_transform = chm_src.transform

In [ ]:
!pip install rioxarray

In [ ]:
import rioxarray as rxr
da_chm = rxr.open_rasterio(chm_path).squeeze().drop(labels='band')
nodata = da_chm.rio.nodata
da_chm = da_chm.where(da_chm != nodata)

print(da_chm)
print(da_chm.dtype)
print(da_chm.values)

<xarray.DataArray (y: 22504, x: 18255)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * x            (x) float64 6.609e+05 6.609e+05 6.609e+05 ... 6.61e+05 6.61e+05
  * y            (y) float64 3.613e+06 3.613e+06 ... 3.613e+06 3.613e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_MAXIMUM:        10.303554534912
    STATISTICS_MEAN:           0.62562006271238
    STATISTICS_MINIMUM:        -0.16918563842773
    STATISTICS_STDDEV:         1.0207351097969
    STATISTICS_VALID_PERCENT:  88.4
    _FillValue:                -3.4028235e+38
    scale_factor:              1.0
    add_offset:                0.0
float32
[[nan nan nan ... nan nan nan]
 [nan nan n

In [ ]:
mean_height = np.mean(chm_array[chm_array > 0])  # Mean height, excluding no-data values
max_height = np.max(chm_array)
min_height = np.min(chm_array[chm_array > 0])  # Min height, excluding no-data values

print(f"Mean Height: {mean_height} meters")
print(f"Max Height: {max_height} meters")
print(f"Min Height: {min_height} meters")

Mean Height: 0.6356552839279175 meters
Max Height: 10.324043273925781 meters
Min Height: 1.9073486328125e-06 meters


In [ ]:
# Mask to identify where tree height is greater than 5 meters
trees_mask = chm_array > 5

# Count the number of pixels representing trees taller than 5 meters
trees_count = np.sum(trees_mask)

print(f"Number of pixels representing trees taller than 5 meters: {trees_count}")

Number of pixels representing trees taller than 5 meters: 1131417
